In [1]:
%load_ext rpy2.ipython
track_file <- "C:/Users/Packr1/Documents/Personal/Track/track_res.csv"
track_frm  <- read_csv(track_file)

ERROR: Error in parse(text = x, srcfile = src): <text>:1:1: unexpected input
1: %load_ext rpy2.ipython
    ^


In [2]:
%lsmagic

ERROR: Error in parse(text = x, srcfile = src): <text>:1:1: unexpected input
1: %lsmagic
    ^


In [4]:
# World Masters Athletics (WMA) Age-graded track tables can be found on
# Howard Grubb's website:
# http://www.howardgrubb.co.uk/athletics/data/wavacalc06.xls

library(rvest)
library(dplyr)
library(tidyr)
library(readxl)
library(lubridate)
library(patchwork)

setwd('C:/Users/Packr1/Documents/Personal/Track')
WMA_Women <- read_xls('wavacalc06.xls', sheet = 1) %>%
              select(2:ncol(.)) %>%
              mutate(`dist(km)_2` = case_when(
                      grepl('K RUN', toupper(Event)) ~ as.numeric(str_extract(Event, "[\\d]*")),
                      TRUE ~ `dist(km)`),
                     dist_m = case_when(
                      grepl("WALK",  toupper(Event)) ~ ((1000 * `dist(km)_2`) + 0.11),
                      TRUE ~ 1000 * `dist(km)_2`)) %>%
              mutate(Sex = 'F') %>%
              # dist_m is 0 for field events 
              # Unknown functional difference between Road distinctions
              #   (e.g., 5kmRoad vs. 5km)
              dplyr::filter(dist_m > 0 & 
                            !grepl('Road', Event))
              print(nrow(WMA_Women) - nrow(WMA_Women %>% distinct(dist_m)))
              message("Expecting 0 so no duplicate distance")
WMA_Men   <- read_xls('wavacalc06.xls', sheet = 2) %>%
              select(2:ncol(.)) %>%
              mutate(dist_m = case_when(
                grepl("WALK",  toupper(Event)) ~ ((1000 * `dist(km)`) + 0.11),
                TRUE ~ 1000 * `dist(km)`)) %>%
              mutate(Sex = 'M') %>%
              # dist_m is 0 for field events 
              # Unknown functional difference between Road distinctions
              #   (e.g., 5kmRoad vs. 5km)
              dplyr::filter(dist_m > 0 & 
                            !grepl('Road', Event))
              print(nrow(WMA_Men) - nrow(WMA_Men %>% distinct(dist_m)))
              message("Expecting 0 so no duplicate distance")
track_file <- "C:/Users/Packr1/Documents/Personal/Track/track_res.csv"
track_res  <- read_csv(track_file) %>%
            # Adding a bit of distance for Walks, to differentiate
            # in join with the age-graded tables
            mutate(dist_m = case_when(
                grepl("WALK", toupper(Event)) ~ as.integer(dist_m + 0.11),
                TRUE ~ dist_m))

Parsed with column specification:
cols(
  Name = col_character(),
  Sex = col_character(),
  Age = col_integer(),
  Time = col_character(),
  ct_evt = col_integer(),
  Event = col_character(),
  mins = col_integer(),
  secs = col_character(),
  track_time = col_double(),
  temp_dist = col_character(),
  dist_m = col_integer(),
  Place = col_integer(),
  Max_Hamlyn_pts = col_integer(),
  Date_Meet = col_date(format = ""),
  agegrp = col_character()
)
